In [1]:
from a_protocol_0.utils.decorators import log

class Test():
    
    @log
    def test_log(self, a, b, c=2, d=None):
        print("exec test_log")
        
t = Test()

t.test_log("toto", "titi", c="42")

AttributeError: 'NoneType' object has no attribute 'log_debug'

In [ ]:
import time

start_at = time.time()
time.sleep(1.2323)
end_at = time.time()

print("%.3f" % (start_at - end_at))

In [ ]:
import traceback

try:
    raise RuntimeError("hello")
except (Exception, RuntimeError):
    print(traceback.format_exc().splitlines())


In [ ]:
"toto".splitlines()

In [ ]:
"%s terminated in %.3fs" % ("toto", 2.3232)

In [ ]:
from functools import partial
from collections import deque, Iterable


class Toto():
    pass

t = Toto()
t.titi = 2

setattr(t, "titi", 3)

f = partial(setattr, t, "titi", 4)
f()

print(t.titi)



isinstance(lambda: True, Iterable)

In [ ]:
from collections import deque, Iterable
steps = deque([1, 2, 3])
steps[-1]

print(0 or 0)

In [2]:
from functools import partial
from a_protocol_0.sequence.Sequence import Sequence

from a_protocol_0 import Protocol0, EmptyModule
p0 = Protocol0(EmptyModule(is_false=False), init_song=False)

def seq_creator(i):
    if i == 0:
        return None
    return Sequence().add(partial(seq_creator, i - 1), name="seq %s" % i).done()

with p0.component_guard():
    seq = Sequence(name="test seq")
    seq.add(partial(seq_creator, 2))
    seq.done()()
    
    print("-------")
    print(bool(seq._current_step._seq))
    print(seq._current_step._seq)
    print(seq._current_step._seq is not None)




P0 - info: Protocol0 script loaded
P0 - info: seq (test seq) : exec step partial.seq_creator
P0 - info: seq (<ipython-input-2-a679d3d527eb>.seq_creator) : exec step seq 2
P0 - info: seq (<ipython-input-2-a679d3d527eb>.seq_creator) : exec step seq 1
P0 - debug: seq (<ipython-input-2-a679d3d527eb>.seq_creator) _is_condition_seq : False (sequence\Sequence.py:137 in _terminate)
P0 - debug: seq (<ipython-input-2-a679d3d527eb>.seq_creator) (sequence\Sequence.py:138 in _terminate)
P0 - debug: [seq (<ipython-input-2-a679d3d527eb>.seq_creator), seq (test seq)] (sequence\Sequence.py:139 in _terminate)
P0 - info: seq (<ipython-input-2-a679d3d527eb>.seq_creator) terminated successfully
P0 - debug: seq (<ipython-input-2-a679d3d527eb>.seq_creator) _is_condition_seq : False (sequence\Sequence.py:137 in _terminate)
P0 - debug: seq (test seq) (sequence\Sequence.py:138 in _terminate)
P0 - debug: [seq (test seq)] (sequence\Sequence.py:139 in _terminate)
P0 - info: seq (<ipython-input-2-a679d3d527eb>.seq_

In [ ]:
d = {
    "a": True,
    "b": False
}

for k, v in d.items():
    print(k, v)